In [1]:
import pyspark
import findspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

In [4]:
spark

In [5]:
arquivo = "files\programacoes.json"

In [6]:
df = spark.read.format('org.apache.spark.sql.json') \
        .load(arquivo)

In [7]:
df = spark.read\
.option("multiline", "true")\
.json(arquivo)
display(df)

DataFrame[__v: bigint, _id: struct<$oid:string>, consumo: struct<programado:array<struct<BA1:bigint,BA2:bigint,BA3:bigint,BA4:bigint,MG1:bigint,MG2:bigint,SP1:bigint,SP2:bigint>>,solicitado:array<struct<AL:bigint,BA1:bigint,BA2:bigint,BA3:bigint,BA4:bigint,BH1:double,BH2:double,MG1:bigint,MG2:bigint,RJ1:double,RN1:bigint,RN2:bigint,SP1:double,SP2:double,SP3:bigint,SP4:bigint>>>, contrato: struct<cliente:struct<apelido:string,cnpj:string,codigoSAP:string,createdAt:string,id:string,nome:string,updatedAt:string>,createdAt:string,deletedAt:string,fimVigencia:string,id:string,inicioVigencia:string,isDeleted:boolean,limiteLancamento:struct<diaFimAnual:string,diaFimMensal:bigint,diaFimSemanal:bigint,diaFimTrimestral:string,diaInicioAnual:string,diaInicioMensal:bigint,diaInicioSemanal:bigint,diaInicioTrimestral:string,horarioDiaria:string,horarioIntradiaria:string>,numero:string,pontoEntrega:array<string>,qdc:double,status:string,tipoGas:string,tipoProgramacao:array<string>,updatedAt:string>, 

In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [ ]:
# Flatten array of structs an structs

def flatten(df):
    # Compute complex filds (lists and structs) in schema
    complex_fields = dict([(field.name, field.dataType)
			for field in df.schema.fields
			if type(field.dataType) == ArrayType or type(field.dataType) == StructType])
    while len(complex_fields)!=0:
        col_name = list(complex_fields.keys())[0]
        print ("Processing :"+col_name+" Type : "+str(type(complex_fields[col_name])))

    # if structType then convert all sub element to columns.
    # i.e. flatten structs
    if (type(complex_fields[col_name]) == StructType):
        expanded = [col(col_name+'.'+k).alias(col_name+'_'+k) for k in [n.name for n in complex_fields[col_name]]]
        df = df.select("*", *expanded).drop(col_name)

    # if arrayType then add the array elements as rows using the explode function
    # i.e. explede arrays
    elif (type(complex_fields[col_name]) == ArrayType):
        df = df.withColumn(col_name, explode_outer(col_name))

    # Recompute remaining complex fields in schema
    complex_fields = dict([(field.name, field.dataType)
			for field in df.schema.fields
			if type(field.dataType) == ArrayType or type(field.dataType) == StructType])
    return df

In [ ]:
df_flatten = flatten(df)